# Producer

In [1]:
import kafka
from kafka       import KafkaProducer
from kafka.admin import KafkaAdminClient
from kafka.admin import NewTopic

from tqdm        import tqdm

import pandas as pd
import os
import boto3
import json
import time

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
KAFKA_HOME = '/usr/local/kafka'
KAFKA_BOOTSTRAP_SERVERS = ['pd-slave3:9092']

In [3]:
# producer definition from IP address given before
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

# connect to the cluster to run admin functions
kafka_admin = KafkaAdminClient(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    api_version=(0,10,2))

In [7]:
if 'topic_stream' in kafka_admin.list_topics():
    kafka_admin.delete_topics(['topic_stream'])
    
if 'topic_results' in kafka_admin.list_topics():
    kafka_admin.delete_topics(['topic_results'])
    
kafka_admin.list_topics()

['__consumer_offsets']

In [8]:
stream_topic = NewTopic(name='topic_stream', 
                        num_partitions=1, 
                        replication_factor=1)

results_topic = NewTopic(name='topic_results', 
                         num_partitions=1, 
                         replication_factor=1)

kafka_admin.create_topics(new_topics=[stream_topic, results_topic])

TopicAlreadyExistsError: [Error 36] TopicAlreadyExistsError: Request 'CreateTopicsRequest_v3(create_topic_requests=[(topic='topic_stream', num_partitions=1, replication_factor=1, replica_assignment=[], configs=[]), (topic='topic_results', num_partitions=1, replication_factor=1, replica_assignment=[], configs=[])], timeout=30000, validate_only=False)' failed with response 'CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='topic_stream', error_code=36, error_message="Topic 'topic_stream' already exists."), (topic='topic_results', error_code=0, error_message=None)])'.

In [10]:
# load data from s3 bucket
# the keys are useless as verify is false
s3_client = boto3.client('s3',
                         endpoint_url='https://cloud-areapd.pd.infn.it:5210',
                         aws_access_key_id='1a4543841b844a88bb3f2eba45764d61',
                         aws_secret_access_key='42e2f16592f54668b8421ecf5ca7ba51',
                         verify=False)

list_obj_contents = s3_client.list_objects(Bucket= 'mapd-minidt-stream')['Contents']

# wait time to have ~1000 rows/s
wait_time = .00018
#wait_time = .0

for i in range(0, len(list_obj_contents)):
    file_name = list_obj_contents[i]['Key']
    row_data = pd.read_csv(s3_client.get_object(Bucket='mapd-minidt-stream', 
                                                Key=file_name).get('Body'), dtype='str')
    # convert file to json
    json_data = row_data.to_dict( 'records' )
    
    # tqdm to visualize progresses
    for msg in tqdm(json_data): 
        producer.send('topic_stream', json.dumps(msg).encode('utf-8'))
        time.sleep(wait_time)
    producer.flush()

 91%|█████████████████████████████████████████    | 1197816/1310720 [19:12<01:48, 1043.93it/s]Exception ignored in: <generator object tqdm.__iter__ at 0x7f9d984e0f20>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 1210, in __iter__
  File "/opt/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 1289, in close
    self._decr_instances(self)
  File "/opt/anaconda3/lib/python3.9/site-packages/tqdm/std.py", line 604, in _decr_instances
    instances = list(filter(
  File "/opt/anaconda3/lib/python3.9/_weakrefset.py", line 70, in __iter__
    yield item
  File "/opt/anaconda3/lib/python3.9/_weakrefset.py", line 33, in __exit__
    w._commit_removals()
  File "/opt/anaconda3/lib/python3.9/_weakrefset.py", line 60, in _commit_removals
    return
KeyboardInterrupt: 


KeyboardInterrupt: 